In [17]:
!pip install tweepy

  Using cached tweepy-3.10.0-py2.py3-none-any.whl (30 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)


In [18]:
!pip install snscrape

In [19]:
#Let's import the libraries that are required here.
import pandas as pd
import tweepy
import csv

In [20]:
#Provide your own credentials here.
consumer_key = '3AhcLhaokcclwer5wmCwzHGT7'
consumer_secret = 'DFtxwduGh3WunYWotO4MJ6H5scRLNDgrBENnwUlY2UgMMnOgS7'
access_token = '1329337544715104256-RKVbgzxG23w2YJOvZe2h2emW7xdJDc'
access_token_secret = 'bLHnH5uqS6ehAMX6PI1MjJ66QGStw7pkAckmV8gQtsSsr'

In [21]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [22]:
tweet_url = pd.read_csv("scraped_tweets.txt", index_col= None, header = None, names = ["tweet_urls"])
tweet_url.head()

,tweet_urls
0,https://twitter.com/hayayaba/status/1364726169...
1,https://twitter.com/nedelle_torrisi/status/136...
2,https://twitter.com/HayleyHolme/status/1364725...
3,https://twitter.com/DanJack45260569/status/136...
4,https://twitter.com/SOEKOCCESS/status/13647252...


In [23]:
#Extract the tweet id
af = lambda x: x["tweet_urls"].split("/")[-1]
#store tweet id in another column
tweet_url['tweet_id'] = tweet_url.apply(af, axis=1)

In [24]:
tweet_url.head()


,tweet_urls,tweet_id
0,https://twitter.com/hayayaba/status/1364726169...,1364726169329823744
1,https://twitter.com/nedelle_torrisi/status/136...,1364726109389025282
2,https://twitter.com/HayleyHolme/status/1364725...,1364725498920714240
3,https://twitter.com/DanJack45260569/status/136...,1364725401608736776
4,https://twitter.com/SOEKOCCESS/status/13647252...,1364725210667229184


In [25]:
#Then, let’s convert our tweet_url Series into a list.
ids = tweet_url['tweet_id'].tolist()

In [26]:
#When I looped through all the 3400 ids, the API is giving an error. Therefore, I tried to process the ids by batch or chunks.
total_count = len(ids)
chunks = (total_count - 1) // 50 + 1
chunks

112

In [27]:
#we can finally create our function that would extract all the elements that we need from the tweet. 
#Here, only the tweet id, tweet time and the tweet itself is required.
def fetch_tw(ids):
    tw_statuses = api.statuses_lookup(ids, tweet_mode= "extended")
    data = pd.DataFrame()
    for status in tw_statuses:
            tweet_elem = {"tweet_id": status.id,
                     "tweet":status.full_text,
                     "date":status.created_at}
            data = data.append(tweet_elem, ignore_index = True)
    data.to_csv("scraped_tweets.csv", mode="a")

In [29]:
#We'll process only 50 entries because the statuses_lookup() method gives the statuses of IDs, up to 100 only..
for i in range(chunks):
        lst = ids[i*50:(i+1)*50]
        result = fetch_tw(lst)